In [1]:
import torch 
import torch.optim.lr_scheduler as schedulers
import matplotlib.pyplot as plt
import numpy as np

from utils import train_batch_step, retrieve_training_params, retrieve_setup

/Users/juangarcia/anaconda3/envs/ada/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/juangarcia/anaconda3/envs/ada/lib/python3.9/site-packages/torchvision/image.so, 6): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: /Users/juangarcia/anaconda3/envs/ada/lib/python3.9/site-packages/torchvision/image.so
  Expected in: /Users/juangarcia/anaconda3/envs/ada/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib
 in /Users/juangarcia/anaconda3/envs/ada/lib/python3.9/site-packages/torchvision/image.so'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/juangarcia/anaconda3/envs/ada/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register

In [2]:
model, dataset = retrieve_setup("MobileNetV3Small", "CIFAR10")
dataset_name = dataset["train"].root.split("/")[-1]

Files already downloaded and verified
Files already downloaded and verified


In [3]:
optimizer, lr_schedulers, loss_fn, n_epochs, batch_size = retrieve_training_params(model, dataset_name)

data_loader = torch.utils.data.DataLoader(dataset["train"], batch_size, shuffle=True)

#total_iters = len(data_loader) * n_epochs   # if updating at every iteration
total_iters = int(len(data_loader) * n_epochs / 10) # if updating every 10 iteration
scheduler = schedulers.LinearLR(optimizer, start_factor=0.1, end_factor=1, total_iters=total_iters)

In [ ]:


# storage
losses = []

# training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
iter = 0
for epoch in range(n_epochs):
    for X_batch, y_batch in data_loader:
        iter += 1
        X_batch.to(device)

        # train with batch
        loss = train_batch_step(X_batch, y_batch, model, optimizer, loss_fn)
        losses.append(loss)

        # update scheduler
        if iter % 10 == 0:
          scheduler.step()

losses = np.array(losses)